# Inspect misclassified examples - training split

This notebook gathers all misclassified items from the chosen model and split, joins them with their titles and abstracts, displays an interactive table, and writes a clean PDF report for human review.

**Steps**

1. Set `MODEL_NAME` if it differs from the default.
2. Run all cells from top to bottom.
3. The PDF will appear in `outputs/<MODEL_NAME>/train/misclassified/misclassified_report.pdf`.

The notebook installs *reportlab* automatically if it is not already present.

In [ ]:
# Cell 1 – Imports and package check
import subprocess, sys

def pip_install(pkg):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

pip_install("reportlab")

import json, shutil
from pathlib import Path
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
print("Libraries ready")

In [ ]:
# Cell 2 – Parameters
MODEL_NAME = "gpt-4.1"   # change if needed
SPLIT = "train"          # keep as 'train' for this task

base_dir = Path("outputs")
predictions_dir = base_dir / MODEL_NAME / SPLIT / "predictions"
misclassified_dir = base_dir / MODEL_NAME / SPLIT / "misclassified"
unparsed_dir = base_dir / MODEL_NAME / SPLIT / "unparsed"

DATASET_DIR  = Path.cwd().parent / "datasets"  
dataset_path = DATASET_DIR / f"{SPLIT}_dataset.csv" 

if not dataset_path.exists():
    raise FileNotFoundError(f"Dataset not found: {dataset_path}")
if not predictions_dir.exists():
    raise FileNotFoundError(f"Folder not found: {predictions_dir}")

print("Dataset file:", dataset_path.resolve())

In [ ]:
# Cell 2.5 - If misclassified folder does not exist, create it

if unparsed_dir.exists():
    unparsed_files = []
    for fp in unparsed_dir.glob("*.json"):
        unparsed_files.append(fp.name)

if not misclassified_dir.exists():
    misclassified_dir.mkdir()
    for fp in predictions_dir.glob("*.json"):
        with open(fp, encoding="utf-8") as f:
            data = json.load(f)
        if (data.get("ground_truth").get("domain") != data.get("prediction").get("domain")) and (fp.name not in unparsed_files):
            shutil.copy(fp, misclassified_dir / fp.name)

print("Misclassified folder:", misclassified_dir.resolve())

In [ ]:
# Cell 3 – Load misclassified JSON files
rows = []
for fp in misclassified_dir.glob("*.json"):
    with open(fp, encoding="utf-8") as f:
        data = json.load(f)

    ground_truth = data.get("ground_truth").get("domain")
    if not ground_truth:
        ground_truth = data.get("ground_truth").get("classification")
    
    prediction = data.get("prediction").get("domain")
    rationale = data.get("prediction").get("domain_rationale")
    if not prediction:
        prediction = data.get("prediction").get("classification")
        rationale = data.get("prediction").get("classification_rationale")

    rows.append({
        "id": fp.stem,
        "ground_truth": ground_truth,
        "prediction": prediction,
        "rationale": rationale
    })

mis_df = pd.DataFrame(rows)
print("Loaded misclassified rows:", len(mis_df))

In [ ]:
# Cell 4 – Merge with title and abstract
dataset_df = pd.read_csv(dataset_path, usecols=["id", "Title", "abstract"])
report_df = mis_df.merge(dataset_df, on="id", how="left")
missing = report_df["abstract"].isna().sum()
if missing:
    print(f"Warning, {missing} abstracts missing after merge")
report_df.head()

In [ ]:
# Cell 5 – Optional preview in notebook
try:
    from ace_tools import display_dataframe_to_user
    display_dataframe_to_user("Misclassified examples", report_df)
except Exception as e:
    print("Interactive display not available:", e)

In [ ]:
# Cell 6 – Create PDF report
styles = getSampleStyleSheet()
normal = styles["Normal"]
bold = ParagraphStyle("Bold", parent=normal, fontName="Helvetica-Bold", fontSize=12, spaceAfter=6)

output_pdf = misclassified_dir / "misclassified_report.pdf"
doc = SimpleDocTemplate(str(output_pdf), pagesize=letter,
                        leftMargin=40, rightMargin=40, topMargin=40, bottomMargin=40)

elements = []
for idx, row in report_df.iterrows():
    try:
        elements.append(Paragraph(f"Example {idx+1} of {len(report_df)}", bold))
        elements.append(Paragraph(f"<b>ID:</b> {row['id']}", normal))
        elements.append(Paragraph(f"<b>Ground truth:</b> {row['ground_truth']}   <b>Prediction:</b> {row['prediction']}", normal))
        elements.append(Paragraph(f"<b>Rationale:</b> {row['rationale']}", normal))
        elements.append(Spacer(1, 8))
        elements.append(Paragraph(f"<b>Title:</b> {row['Title']}", normal))
        elements.append(Spacer(1, 4))
        elements.append(Paragraph(f"<b>Abstract:</b> {row['abstract']}", normal))
        elements.append(PageBreak())
    except:
        print("Error encountered with ", row["id"])
        elements.append(PageBreak())
        continue

doc.build(elements)
print("PDF saved to:", output_pdf.resolve())